In [1]:
!pip install albumentations==0.5.2
!pip install efficientnet_pytorch
!pip install pycocotools
!pip install cython
!pip install timm
!pip install effdet
!pip install gluoncv
!pip install mxnet

     |████████████████████████████████| 111 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 14.1 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 42.1 MB/s eta 0:00:01
     |████████████████████████████████| 334 kB 41.1 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 960 kB 41.1 MB/s eta 0:00:01
     |████████████████████████████████| 699 kB 59.4 MB/s eta 0:00:01
     |████████████████████████████████| 950 kB 54.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 111 kB 55.2 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 273 kB 54.8 MB/s eta 0:00:01
  Created wheel for autograd: filename=autograd-1

In [2]:
from pycocotools.coco import COCO
import gluoncv.utils as gcv
import torch
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import warnings
from torch.optim.lr_scheduler import _LRScheduler
import math
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [4]:
batch_size = 4
learning_rate = 1e-4
num_epochs = 200
seed = 41
seed_everything(seed)

In [5]:
IMAGENET_DEFAULT_MEAN = [x * 255 for x in (0.485, 0.456, 0.406)]
IMAGENET_DEFAULT_STD = [x * 255 for x in (0.229, 0.224, 0.225)]

class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''
    def __init__(self, annotation, data_dir, transforms):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image, boxes, labels = self.load_image_and_boxes(index)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        
        # Multi Scale
        target['img_size'] = torch.tensor([(512, 512)])
        target['img_scale'] = torch.tensor([1.])
        
        if self.transforms:
            # Transform 적용 후 Box가 없다면 있을 때 까지 반복 (default = 10)
            for i in range(10):
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': target['labels']
                }
                sample = self.transforms(**sample)
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  #yxyx: be warning
                    target['labels'] = torch.tensor(sample['labels'])
                    break
        
        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())
    
    def load_image_and_boxes(self, index):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = (image - IMAGENET_DEFAULT_MEAN) / IMAGENET_DEFAULT_STD

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])
        
        labels = np.array([x['category_id'] for x in anns])
        #labels = torch.as_tensor(labels, dtype=torch.int64)

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        return image, boxes, labels

In [10]:
def collate_fn(batch):
    return tuple(zip(*batch))

def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

def get_valid_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels'])
    )

train_annotation = '/opt/ml/input/data/train.json'
val_annotation = '/opt/ml/input/data/val.json'
data_dir = '/opt/ml/input/data'
train_dataset = CustomDataset(train_annotation, data_dir, get_train_transform())
val_dataset = CustomDataset(val_annotation, data_dir, get_valid_transform())

train_data_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=0)
val_data_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=0)

loading annotations into memory...
Done (t=4.08s)
creating index...
index created!
loading annotations into memory...
Done (t=0.87s)
creating index...
index created!


In [11]:
def get_model():
    # get weight
    # url : https://github.com/rwightman/efficientdet-pytorch
    config = get_efficientdet_config('tf_efficientdet_d4')
    config.image_size = (512, 512)
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    net = EfficientDet(config, pretrained_backbone=True)
    checkpoint = torch.load('./pretrained_weight/tf_efficientdet_d4_49-f56376d9.pth')
    net.load_state_dict(checkpoint)

    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=11)
    return DetBenchTrain(net, config)

In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class EarlyStopping:
    def __init__(self, patience=100, verbose=False, delta=0, path=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.path = path

    def __call__(self, val_mAP, model):
        score = val_mAP
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        model.eval()
        torch.save(model.state_dict(), self.path)

In [13]:
# Use freeze : model.apply(freeze_backbone)
# Use Unfreeze : model.apply(Unfreeze_backbone)
def freeze_backbone(m):
    classname = m.__class__.__name__
    for ntl in ['EfficientNet', 'BiFPN']:
        if ntl in classname:
            for param in m.parameters():
                param.requires_grad = False

def Unfreeze_backbone(m):
    classname = m.__class__.__name__
    for ntl in ['EfficientNet', 'BiFPN']:
        if ntl in classname:
            for param in m.parameters():
                param.requires_grad = True

In [14]:
def train_fn(model, train_data_loader, val_data_loader, optimizer, device):
    model.train()
    train_loss = AverageMeter()
    train_cls_loss = AverageMeter()
    train_box_loss = AverageMeter()
    save_path = './saved/checkpoint.pth'
    early_stop = EarlyStopping(path=save_path)
    for epoch in range(num_epochs):
        train_loss.reset()
        train_cls_loss.reset()
        train_box_loss.reset()
        model.train()
        for images, targets, image_ids in tqdm(train_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()
            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(512, 512) for target in targets]).to(device).float()

            # calculate loss
            losses, cls_loss, box_loss = model(images, targets2).values()

            train_loss.update(losses.detach().item(), current_batch_size)
            train_cls_loss.update(cls_loss.detach().item(), current_batch_size)
            train_box_loss.update(box_loss.detach().item(), current_batch_size)

            # backward
            optimizer.zero_grad()
            losses.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)
            optimizer.step()
        
        # Evaluation
        val_loss, val_mAP = vali_fn(val_data_loader,model,device)
        
        print(f"\nEpoch #{epoch+1} train loss: [{train_loss.avg}] train cls loss : [{train_cls_loss.avg}] train box loss : [{train_box_loss.avg}] validation mAP : [{val_mAP}] \n")
        wandb.log({"Train Loss":train_loss.avg, "Validation mAP@50":val_mAP})
        early_stop(val_mAP,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [15]:
def vali_fn(val_data_loader, model, device):
    model.eval()
    vali_loss = AverageMeter()
    vali_mAP = AverageMeter()
    # Custom
    metric = gcv.metrics.VOCMApMetric(iou_thresh=0.5)
    with torch.no_grad():
        for images, targets, image_ids in tqdm(val_data_loader):
            # gpu 계산을 위해 image.to(device)
            images = torch.stack(images)
            images = images.to(device).float()

            current_batch_size = images.shape[0]

            targets2 = {}
            targets2['bbox'] = [target['boxes'].to(device).float() for target in targets] # variable number of instances, so the entire structure can be forced to tensor
            targets2['cls'] = [target['labels'].to(device).float() for target in targets]
            targets2['image_id'] = torch.tensor([target['image_id'] for target in targets]).to(device).float()
            targets2['img_scale'] = torch.tensor([target['img_scale'] for target in targets]).to(device).float()
            targets2['img_size'] = torch.tensor([(512, 512) for target in targets]).to(device).float()

            outputs = model(images, targets2)

            loss = outputs['loss']
            det = outputs['detections']

            # Calc Metric
            for i in range(0, len(det)):
                pred_scores=det[i, :, 4].cpu().unsqueeze_(0).numpy()
                condition=(pred_scores > 0.05)[0]
                gt_boxes=targets2['bbox'][i].cpu().unsqueeze_(0).numpy()[...,[1,0,3,2]] #move to PASCAL VOC from yxyx format
                gt_labels=targets2['cls'][i].cpu().unsqueeze_(0).numpy()

                pred_bboxes=det[i, :, 0:4].cpu().unsqueeze_(0).numpy()[:, condition, :]
                pred_labels=det[i, :, 5].cpu().unsqueeze_(0).numpy()[:, condition]
                pred_scores=pred_scores[:, condition]
                metric.update(
                  pred_bboxes=pred_bboxes,
                  pred_labels=pred_labels,
                  pred_scores=pred_scores,
                  gt_bboxes=gt_boxes,
                  gt_labels=gt_labels)

            vali_mAP.update(metric.get()[1], current_batch_size)
            vali_loss.update(loss.detach().item(), current_batch_size)
    
    # validation loss
    return vali_loss.avg, vali_mAP.avg

In [17]:
# Model
model = get_model()
model.to(device)
model.eval()
# setting
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=learning_rate)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b4_aa-818f208c.pth" to /opt/ml/.cache/torch/hub/checkpoints/tf_efficientnet_b4_aa-818f208c.pth


FileNotFoundError: [Errno 2] No such file or directory: './pretrained_weight/tf_efficientdet_d4_49-f56376d9.pth'

In [16]:
train_fn(model, train_data_loader, val_data_loader, optimizer, device)

NameError: name 'model' is not defined